In [ ]:
# 导入必要的库
import pandas as pd          # 数据操作库，核心函数：pd.read_csv()读取CSV文件
import numpy as np           # 数值计算库，提供多维数组支持
import matplotlib.pyplot as plt  # 绘图库，plt.plot()用于绘制图表
import seaborn as sns        # 高级可视化库，sns.heatmap()绘制热力图
from sklearn.model_selection import (
    train_test_split,        # 数据集划分函数，参数：test_size, random_state, stratify
    GridSearchCV             # 超参数网格搜索类，参数：estimator, param_grid, cv, scoring
)
from sklearn.preprocessing import (
    StandardScaler,          # 数值标准化（均值0，方差1），用于缩放特征
    OrdinalEncoder           # 分类特征编码，将类别标签转为整数（例如性别转0/1）
)
from sklearn.compose import ColumnTransformer  # 多列不同预处理的组合工具，参数：transformers, remainder
from sklearn.pipeline import Pipeline          # 流水线化处理流程，参数：steps
from sklearn.ensemble import RandomForestClassifier  # 随机森林分类器，参数：n_estimators, max_depth等
from sklearn.metrics import (                  # 模型评估指标
    accuracy_score,          # 准确率 = (TP+TN)/(TP+FP+TN+FN)
    precision_score,         # 精确率 = TP/(TP+FP)
    recall_score,            # 召回率 = TP/(TP+FN)
    f1_score,                # F1 = 2*(Precision*Recall)/(Precision+Recall)
    roc_auc_score,           # ROC曲线下面积，衡量分类器整体性能
    roc_curve                # 计算ROC曲线的真阳性率和假阳性率
)

# 加载数据
# pd.read_csv()参数说明：
# - filepath: 文件路径（字符串）
# - sep: 分隔符（默认','），例如读取TSV文件需指定sep='\t'
# - header: 指定列名所在行（默认0，即第一行）
df = pd.read_csv('/kaggle/input/houkong-moai/customer_churn.csv')  # 读取客户流失数据集

# 定义预处理流水线
# ColumnTransformer参数说明：
# - transformers: 预处理步骤列表，每个元素是元组（名称, 转换器, 列）
# - remainder: 未被处理的列处理方式（'passthrough'表示保留原数据）
preprocessor = ColumnTransformer(
    transformers=[
        # 对数值列标准化处理
        # - 'num': 自定义步骤名称
        # - StandardScaler(): 转换器对象（标准化：均值=0，标准差=1）
        # - ['age', ...]: 要处理的数值列列表
        ('num', StandardScaler(), ['age', 'subscription_length', 'monthly_bill', 'total_usage', 'service_complaints']),
        
        # 对分类列编码
        # - 'cat': 步骤名称
        # - OrdinalEncoder(): 将性别（如Male/Female）转为0/1
        # - ['gender']: 分类列名列表
        ('cat', OrdinalEncoder(), ['gender'])
    ],
    remainder='passthrough'  # 其他列（若有）直接保留，不处理
)

# 定义特征矩阵和标签
# df.drop()参数说明：
# - labels: 要删除的列名列表（['customer_id', 'churn']）
# - axis=1: 按列删除（axis=0为行）
X = df.drop(['customer_id', 'churn'], axis=1)  # 特征矩阵（移除ID和标签列）
y = df['churn']  # 标签列（客户是否流失，0/1）

# 划分训练集和测试集
# train_test_split()参数说明：
# - test_size=0.2: 测试集占比20%
# - random_state=42: 固定随机种子，确保结果可复现
# - stratify=y: 按标签分层抽样，保持训练集和测试集的流失比例一致
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

# 创建模型流水线
# Pipeline参数说明：
# - steps: 处理步骤列表，按顺序执行
# - ('preprocessor', preprocessor): 第一步应用预处理
# - ('classifier', RandomForestClassifier()): 第二步使用随机森林分类器
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# 定义超参数网格
# param_grid结构：
# - 键：参数名（格式为"步骤名__参数名"）
# - 值：候选参数值列表
param_grid = {
    'classifier__n_estimators': [100, 200],      # 随机森林中树的数量（100或200）
    'classifier__max_depth': [None, 10],         # 树的最大深度（None表示不限制，10表示最多10层）
    'classifier__min_samples_split': [2, 5]      # 节点分裂所需最小样本数（2或5）
}

# 执行网格搜索
# GridSearchCV参数说明：
# - estimator=pipeline: 要优化的模型流程
# - param_grid=param_grid: 超参数候选值网格
# - cv=3: 3折交叉验证（训练集分成3份，2份训练，1份验证）
# - scoring='roc_auc': 以ROC AUC作为评估指标（适用于二分类不平衡数据）
grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=3, 
    scoring='roc_auc'
)
grid_search.fit(X_train, y_train)  # 在训练集上拟合模型并搜索最优参数

# 获取最优模型
best_model = grid_search.best_estimator_  # best_estimator_属性返回最优参数组合的模型

# 加载测试数据（需替换为真实测试集路径）
test_data = pd.read_csv('/kaggle/input/houkong-moai/customer_churn.csv')  # 假设测试集结构与训练集相同

# 生成预测结果
# best_model.predict()参数说明：
# - 输入特征矩阵（需与训练集结构一致，移除ID和标签列）
predictions = best_model.predict(test_data.drop(['customer_id', 'churn'], axis=1))

# 创建提交文件
# pd.DataFrame()参数说明：
# - data: 字典格式，列名为'customer_id'和'churn'
# - columns: 指定输出列顺序
submission = pd.DataFrame({
    'customer_id': test_data['customer_id'],  # 客户ID列
    'churn': predictions                      # 预测的流失标签（0/1）
})

# 保存为CSV文件
# to_csv()参数说明：
# - path_or_buf='submission.csv': 输出文件路径
# - index=False: 不保存行索引
submission.to_csv('submission.csv', index=False)

print("提交文件已生成！")  # 输出提示信息

In [ ]:
#代码行	                      函数/类	               关键参数	                参数含义
#pd.read_csv(...)	          pandas.read_csv	       filepath, sep, header	文件路径、分隔符、列名所在行
#ColumnTransformer(...)	      ColumnTransformer	       transformers, remainder	预处理步骤组合、未处理列的保留方式
#train_test_split(...)	      train_test_split	       test_size, stratify	    测试集比例、按标签分层抽样
#Pipeline(...)	              Pipeline	               steps	                流水线步骤列表（名称, 对象）
#GridSearchCV(...)	          GridSearchCV	           param_grid, scoring	    超参数候选值网格、评估指标
#RandomForestClassifier(...)  RandomForestClassifier   n_estimators, max_depth	决策树数量、树的最大深度
#submission.to_csv(...)	      DataFrame.to_csv	       index	                是否保存行索引（设为False避免多余列）
